In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from IPython.display import display, Latex

In [ ]:
def fraction_to_latex(r):
    i,j = (round(r,5)).as_integer_ratio()
    if abs(j) == 1 or i==0:
        return r'$'+str(i)+'$'
    #only fractions left
    if r < 0:
        return r'\frac{'+str(-abs(i))+'}{'+str(abs(j))+'} '
    return r'\frac{'+str(i)+'}{'+str(j)+'} '

In [ ]:
def mult(s):
    return int(2*s + 1)
def m(s):
    return [s-i for i in range(0,mult(s))]
def basis(S):
    allm = list(itertools.product(*[m(s) for s in S]))
    return allm
    #list(itertools.product(*a))
def print_basis(S):
    all_vectors=basis(S)
    for v in all_vectors:
        vector=r'$|'
        for m in v:
            vector+=fraction_to_latex(m)+','
        vector=vector[:-1]+r'>$'
        display(Latex(vector))

In [ ]:
print_basis([1/2,1,1/2])

In [ ]:
a=0
a.as_integer_ratio()

If we have a chain of spins $\vec{S}_i$
\begin{equation}
H=\sum_{\alpha,j>i} J^{\alpha}_{ij} \tilde{S}^{\alpha}_i \tilde{S}^{\alpha}_j \, \alpha=x,y,z \\
\tilde{S}^{\alpha}_k = I^{2*S_1+1}_1\otimes I^{2*S_2+1}_2 \otimes \cdots \otimes I^{2*S_{k-1}+1}_{k-1}\otimes S^{\alpha}_k \otimes \cdots I^{2*S_N+1}_N
\end{equation}
where $I^{2*S_2+1}_2$ is the identity matrix of dimension the multiplicity of $S_2$ and $\otimes$ is the kronecker product

in general:
\begin{equation}
<m'|S^x|m>=(\delta _{m' , m+1} + \delta _{m'+1,m} \frac{1}{2})\sqrt{S(S+1)-mm'} \\
<m'|S^y|m>=(\delta _{m' , m+1} - \delta _{m'+1,m} \frac{1}{2i})\sqrt{S(S+1)-mm'} \\
<m'|S^z|m>=\delta _{m' , m}m
\end{equation}

The functions sx,sy,sz defined below accept as input a list $S$ containing the value of the $N$ spins in the chain and the index $j$ of the spin for which the matrix $S^{\alpha}_j$ should be computed. $sx([1/2,1,1/2],1)$ would return the 12-D matrix of $\tilde{S}^x$ for the spin $j=1$ in a trimer where $j=0$ has $S=1/2$, $j=1$ has $S=1$ and $j=2$ has $S=1/2$

In [ ]:
def delta(m1,m2):
    return int((int(2*m1)==int(2*m2)))
def sx(S,jj):
    SM=[]
    for ii,s in enumerate(S):
        m=[s-i for i in range(0,mult(s))]
        sm=complex(1)*np.zeros((mult(s),mult(s)))
        for i,m1 in enumerate(m):
            for j,m2 in enumerate(m):
                sm[i,j]=1/2*np.sqrt(s*(s+1)-m1*m2)*(delta(m1,m2+1)+delta(m1+1,m2))
        if ii == jj:
            SM.append(sm)
        else:
            SM.append(np.identity(mult(s))) # Identity matrix of 2*S+1 dimension
    sm = SM[0]
    for i in range(1,len(S)): # do I^x(j-1) x s_j x I^x(N-j)
        sm=np.kron(sm,SM[i])
    return sm
def sy(S,jj):
    SM=[]
    for ii,s in enumerate(S):
        m=[s-i for i in range(0,mult(s))]
        sm=complex(1)*np.zeros((mult(s),mult(s)))
        for i,m1 in enumerate(m):
            for j,m2 in enumerate(m):
                sm[i,j]=complex(1/(2j))*np.sqrt(s*(s+1)-m1*m2)*(delta(m1,m2+1)-delta(m1+1,m2))
        if ii == jj:
            SM.append(sm)
        else:
            SM.append(np.identity(mult(s))) # Identity matrix of 2*S+1 dimension
    sm = SM[0]
    for i in range(1,len(S)): # do I^x(j-1) x s_j x I^x(N-j)
        sm=np.kron(sm,SM[i])
    return sm
def sz(S,jj):
    SM=[]
    for ii,s in enumerate(S):
        m=[s-i for i in range(0,mult(s))]
        sm=complex(1)*np.zeros((mult(s),mult(s)))
        for i,m1 in enumerate(m):
            sm[i,i]=m1
        if ii == jj:
            SM.append(sm)
        else:
            SM.append(np.identity(mult(s))) # Identity matrix of 2*S+1 dimension
    sm = SM[0]
    for i in range(1,len(S)): # do I^x(j-1) x s_j x I^x(N-j)
        sm=np.kron(sm,SM[i])
    return sm

for example for $S=1/2$ the $S^{\alpha}$ matrices are:
\begin{equation}
S^x = \frac{1}{2} 
\begin{pmatrix} 
1 & 0 \\
0 & 1
\end{pmatrix}
S^y = \frac{1}{2i} 
\begin{pmatrix} 
0 & 1 \\
-1 & 0
\end{pmatrix}
S^z = \frac{1}{2} 
\begin{pmatrix} 
1 & 0 \\
0 & -1
\end{pmatrix}
\end{equation}

and for S=1 
\begin{equation}
S^x = \frac{1}{\sqrt{2}} \begin{pmatrix} 0 & 1 & 0 \\ 1 & 0 & 1 \\ 0 & 1 & 0 \end{pmatrix} ,
S^y = \frac{1}{\sqrt{2}i}  \begin{pmatrix}   0 & 1 & 0 \\ -1 & 0 & 1 \\ 0 & -1 & 0 \end{pmatrix} , 
S^z = \frac{1}{\sqrt{2}}  \begin{pmatrix}   1 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & -1 \end{pmatrix}
\end{equation}

in case of a dimer with $S_1 = 1/2$ and $S_2=1/2$ we have: 
\begin{equation}
\tilde{S}^z_1 = S^z  \otimes \begin{pmatrix}1 & 0 \\0 & 1 \end{pmatrix} = 
\frac{1}{2}\begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\  0 & 0 & -1 & 0 \\ 0 & 0 & 0 & -1 \end{pmatrix}\\
\tilde{S}^z_2 = \begin{pmatrix}1 & 0 \\0 & 1 \end{pmatrix} \otimes S^z = 
\frac{1}{2}\begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\  0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{pmatrix}
\end{equation}
and
\begin{equation}
\tilde{S}^z_1 \tilde{S}^z_2 = 
\frac{1}{2}\begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\  0 & 0 & -1 & 0 \\ 0 & 0 & 0 & -1 \end{pmatrix} 
\frac{1}{2}\begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\  0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{pmatrix} =
\frac{1}{4} \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\  0 & 0 & -1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix}
\end{equation}
equivalent to
\begin{equation}
S^z \otimes S^z
\end{equation}
as shown also in the four code cells below

In [ ]:
sz([1/2,1/2],0)

In [ ]:
sz([1/2,1/2],1)

In [ ]:
np.matmul(sz([1/2,1/2],0),sz([1/2,1/2],1))

In [ ]:
np.kron(sz([1/2],0),sz([1/2],0))

# example: Hamiltonian two S=1/2 spins

In [ ]:
H=np.matmul(sx([1/2,1/2],0),sx([1/2,1/2],1)) + np.matmul(sy([1/2,1/2],0),sy([1/2,1/2],1)) +np.matmul(sz([1/2,1/2],0),sz([1/2,1/2],1))
H

In [ ]:
evals, evecs = np.linalg.eig(H)

In [ ]:
np.real(evals)

In [ ]:
np.real(evecs)[:,0]

# dimer of S=1/2 and S=1

in case of a dimer with spin $S_1=1/2$ and spin $S_2=1$ we have:
\begin{equation}
\tilde{S}^z_1 = S^z  \otimes \begin{pmatrix}1 & 0 &0  \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}=
\frac{1}{2}\begin{pmatrix} 1 &0 &0 &0 &0 &0 \\ 0 &1 &0 &0 &0 &0 \\ 0 &0 &1 &0 &0 &0 \\ 0 &0 &0 &-1 &0 &0\\ 0 &0 &0 &0 &-1 &0 \\ 0 &0 &0 &0 &0 &-1 \end{pmatrix} \\
\tilde{S}^z_2 = \begin{pmatrix}1 & 0 \\ 0 & 1  \end{pmatrix} \otimes S^z = 
\begin{pmatrix} 1 &0 &0 &0 &0 &0 \\ 0 &0 &0 &0 &0 &0 \\ 0 &0 &-1 &0 &0 &0 \\ 0 &0 &0 &1 &0 &0\\ 0 &0 &0 &0 &0 &0 \\ 0 &0 &0 &0 &0 &-1 \end{pmatrix}
\end{equation}
where we used the $S^z$ matrix for $S=\frac{1}{2}$ in the first equation and for $S=1$ in the second one. This is also demonstarted in the two following code cells.

In [ ]:
sz([1/2,1],0)

In [ ]:
sz([1/2,1],1)

# general case

$J=(J^x_{1,2},J^x_{1,3},\cdots,J^x_{1,N},J^x_{2,3},\cdots,J^x_{N-1,N}),(J^y,\cdots),(J^z,\cdots)$

In [ ]:
J=[[],[],[]]
N=2
jx=1
jy=1
jz=1
count=0
for i in range(N-1):
    for j in range(i+1,N):
        J[0].append(int(i+1==j)*jx)
        J[1].append(int(i+1==j)*jy)
        J[2].append(int(i+1==j)*jz)
    count+=1
S=[1,1]

In [ ]:
def H(S,J):
    ndim=1
    for s in S:
        ndim*=mult(s)
    h=complex(1)*np.zeros((ndim,ndim))
    count=0
    for i in range(len(S)-1):
        for j in range(i+1,len(S)):
            h+=J[0][count]*np.matmul(sx(S,i),sx(S,j))
            h+=J[1][count]*np.matmul(sy(S,i),sy(S,j))
            h+=J[2][count]*np.matmul(sz(S,i),sz(S,j))
            count+=1
    return h

In [ ]:
h=H(S,J)

In [ ]:
evals, evecs = np.linalg.eig(h)

In [ ]:
np.real(evals)

In [ ]:
np.real(evecs)[:,0]